In [1]:
import pandas as pd

print("Loading chunks...")
df = pd.read_parquet("chunks.parquet")
chunks = df.to_dict(orient="records")

print(f"✓ Loaded {len(chunks)} chunks")
print("\nSample chunk:")
print(chunks[0]['chunk_text'][:200])


Loading chunks...
✓ Loaded 1642 chunks

Sample chunk:
Student Guide Welcome to Istanbul Sabahattin Zaim University (IZU) We are delighted to welcome all international students to IZU. To ensure a smooth transition into university life, we highly recommen


In [2]:
import pandas as pd
df = pd.read_parquet("/home/zeynkash/projects/izu_scraper/chunking/chunks.parquet")
print(df.columns)
# ensure the text column name (common names: chunk_text, content, text)
# Example: rename to standard fields:
if 'chunk_text' in df.columns:
    df = df.rename(columns={'chunk_text': 'content'})
elif 'text' in df.columns:
    df = df.rename(columns={'text': 'content'})
# create chunk_id if missing
if 'chunk_id' not in df.columns:
    df['chunk_id'] = df.index.astype(str)
# create a list of dicts with required fields
records = df[['chunk_id', 'content', 'url', 'title']].to_dict(orient='records')
# write JSONL (one JSON object per line) — memory-friendly
with open("chunks.jsonl", "w", encoding="utf-8") as f:
    for r in records:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")
print("Wrote chunks.jsonl with", len(records), "records")


Index(['source_id', 'url', 'title', 'language', 'chunk_id', 'chunk_index',
       'chunk_text', 'chunk_len_chars', 'original_len_chars'],
      dtype='object')


NameError: name 'json' is not defined

In [14]:
import json
import time
from openai import OpenAI
from tqdm import tqdm
import numpy as np

client = OpenAI()  # ensure OPENAI_API_KEY is set in env

embs = []
meta = []
with open("chunks.jsonl", "r", encoding="utf-8") as f:
    for line in tqdm(f):
        obj = json.loads(line)
        text = obj.get("content") or obj.get("chunk_text") or obj.get("text")
        if not text or not str(text).strip():
            continue
        # call OpenAI embeddings
        resp = client.embeddings.create(model="text-embedding-3-small", input=text)
        emb = resp.data[0].embedding
        embs.append(emb)
        meta.append({
            "chunk_id": obj.get("chunk_id"),
            "url": obj.get("url"),
            "title": obj.get("title")
        })
        time.sleep(0.05)  # polite pacing to avoid rate limits

# save
np.save("embeddings.npy", np.array(embs, dtype=np.float32))
with open("metadata.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable